In [1]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
from tqdm.auto import tqdm 

In [2]:
folder = r"parsedData/"

event_name = {
    "0-SLW": 1,
    "1-MLW": 2,
    "2-FLW": 3,
    "3-RD": 4,
    "4-SD": 5,
    "5-sit": 6,
    "6-stand": 7,
    "7-RA": 8,
    "8-SA": 9,
}

# Read and mark the Signal
X3s = []
ys = []

for name, label in event_name.items():
    with open(os.path.join(folder, f"{name}.npy"), "rb") as f:
        mat = np.load(f)
    X3s.append(mat)
    ys.append(np.zeros(mat.shape[0]) + label)

X3 = np.concatenate(X3s)
y = np.concatenate(ys)

# Simple Report
X3.shape, y.shape

((31625, 20, 6), (31625,))